In [ ]:
import os
import sys
from distutils.util import strtobool
from functools import partial
from pathlib import Path
from subprocess import CREATE_NEW_CONSOLE, Popen
from typing import Dict, List, Optional, Set, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize as opt_minimize

In [ ]:
def custom_strtobool(val: str) -> bool:
    """Converts a string to a boolean value."""
    val = val.lower()
    if val in ("y", "yes", "t", "true", "on", "1"):
        return True
    elif val in ("n", "no", "f", "false", "off", "0"):
        return False
    else:
        raise ValueError(f"Недопустимое булево значение: {val}")


def load_Cfg_new(
    path_to_config,
    keys_to_bool: set = None,
    keys_to_int: set = None,
    keys_to_float: set = None,
    keys_to_str: set = None,
    com_line: str = "/////////////////////// | Для коментарів | /////////////////////////",
) -> dict:
    """
    Loads configuration from an INI file using configparser.
    Applies type conversions based on provided key sets.
    """

    _keys_to_bool = keys_to_bool if keys_to_bool is not None else {"Px", "Py", "Pz"}
    _keys_to_int = (
        keys_to_int
        if keys_to_int is not None
        else {
            "Seed",
            "Sx",
            "Sy",
            "Sz",
            "mode",
            "AddI",
            "AddFrom",
            "RemI",
            "RemFrom",
            "LoadPrev",
            "StepLim",
            "PrintI",
            "WriteI",
        }
    )
    _keys_to_float = (
        keys_to_float
        if keys_to_float is not None
        else {
            "T",
            "Ax",
            "Ay",
            "Az",
            "g100",
            "g010",
            "g001",
            "dg",
            "C_eq",
            "C0",
            "N_tot",
            "N0_cr",
            "p_b",
        }
    )
    _keys_to_str = (
        keys_to_str
        if keys_to_str is not None
        else {
            "DirPrefix",
        }
    )

    try:
        cfg_dict = [
            *map(
                lambda line: line.strip(),
                open(path_to_config, mode="r", encoding="utf-8")
                .read()
                .strip()
                .split("\n"),
            )
        ]

        com_line_start_id = cfg_dict.index(com_line)

        cfg_part = [*filter(lambda item: item != "", cfg_dict[:com_line_start_id])]
        com_part = cfg_dict[com_line_start_id:]

        cfg_dict = {}

        for line in cfg_part:
            k, v = line.split(":")
            k, v = k.strip(), v.strip()

            if k in _keys_to_int:
                try:
                    cfg_dict[k] = int(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в целое число. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_float:
                try:
                    cfg_dict[k] = float(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в число с плавающей точкой. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_bool:
                try:
                    cfg_dict[k] = custom_strtobool(v)
                except ValueError:
                    print(
                        f"Ошибка: Не удалось преобразовать значение '{v}' для ключа '{k}' в булево. Проверьте WorkCfg.ini.",
                        file=sys.stderr,
                    )
                    sys.exit(1)
            elif k in _keys_to_str:
                if (v.startswith('"') & v.endswith('"')) | (
                    v.startswith("'") & v.endswith("'")
                ):
                    v = v[1:-1]
                cfg_dict[k] = v
            else:
                cfg_dict[k] = v

        return cfg_dict
    except FileNotFoundError:
        print(
            f"Ошибка: Файл конфигурации '{path_to_config}' не найден.", file=sys.stderr
        )
        sys.exit(1)
    except Exception as e:
        print(
            f"Неизвестная ошибка при загрузке файла конфигурации '{path_to_config}': {e}",
            file=sys.stderr,
        )
        sys.exit(1)


def load_XYZ_from_template(SizeX=10, SizeY=10, SizeZ=10, mode=1):
    try:
        X, Y, Z = np.mgrid[0:SizeX, 0:SizeY, 0:SizeZ]

        if mode == 1:
            mask = (X + Y + Z) % 2 == 0
        elif mode == 2:
            mask = (X + Y + Z) > -1
        else:
            return None, None, None

        X, Y, Z = X[mask], Y[mask], Z[mask]

        return X, Y, Z
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None, None


def count_nonempty_lines(file_path):
    try:
        with open(file_path, "r") as file:
            nonempty_lines = sum(1 for line in file if line.strip())
            return nonempty_lines
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except IOError:
        print(f"Error: Unable to read the file at {file_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
def calculate_r2(y_true, y_pred):
    """
    Обчислює коефіцієнт детермінації (R²) між двома наборами числових даних.

    Параметри:
    y_true (array-like): Реальні (спостережувані) значення.
    y_pred (array-like): Передбачені (модельні) значення.

    Повертає:
    float: Коефіцієнт детермінації R².
    """
    # Перетворення у масиви NumPy
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Середнє значення реальних значень
    y_mean = y_true.mean()

    # Сума квадратів залишків (помилок моделі)
    ss_res = np.sum((y_true - y_pred) ** 2)

    # Загальна сума квадратів (розсіювання відносно середнього)
    ss_tot = np.sum((y_true - y_mean) ** 2)

    # Захист від ділення на нуль (всі значення однакові)
    if ss_tot == 0:
        return 0.0

    # Обчислення коефіцієнта детермінації
    r2 = 1 - (ss_res / ss_tot)

    return r2


# Функція обчислення помилки для параметрів
def model_error(params, x, y, model_func):
    return error_metric(y, model_func(x, *params))


# Апроксимація: перебір моделей та методів оптимізації
def approximate(x, y, models, sort_by=-1, reverse=True):
    results = []
    methods = [
        "Nelder-Mead",
        "Powell",
        "CG",
        "BFGS",
        "L-BFGS-B",
        "TNC",
        "COBYLA",
        "SLSQP",
        "trust-constr",
        # "trust-ncg",
        # "trust-exact",
        # "trust-krylov",
    ]

    for method in methods:
        for model_id, model_data in models.items():
            objective = partial(
                model_data["error"], x=x, y=y, model_func=model_data["func"]
            )
            init = model_data["init_params"]
            bounds = [(None, None)] * len(init)
            result = opt_minimize(objective, init, method=method, bounds=bounds)

            if result.success:
                coeffs = result.x
                r2 = calculate_r2(y, model_data["func"](x, *coeffs))
                results.append([method, model_id, *coeffs, r2])

    # Сортування результатів
    results.sort(key=lambda r: r[sort_by], reverse=reverse)

    # Повернення унікальних моделей (перша найкраща для кожної)
    seen, final = set(), []
    for res in results:
        if res[1] not in seen:
            seen.add(res[1])
            final.append(res)

    return final

In [ ]:
path_to_dir = Path(
    r"1750707812686172_Test1_dst_X50Y400Z50_T3e2_C7.14e-12_Nt1e13_Pb-1.0"
)

cfg = load_Cfg_new(path_to_dir / "InitSettings.ini")
run_id = path_to_dir.name.split("_")[0]
cfg["id"] = run_id

data = (path_to_dir / "sim_history.txt").open(mode="r", encoding="utf-8").read()
data = [
    *map(
        lambda line: [*map(lambda item: float(item), line.split(":"))],
        data.splitlines(),
    )
]
data = np.array(data)
data = {
    "time": data[8],
    "n_gas_history": data[0],
    "n_crystal_history": data[1],
    "concentration_history": data[2],
    "delta_gibbs_history": data[3],
    "energy_change_history": data[4],
    "crystal_sx_history": data[5],
    "crystal_sy_history": data[6],
    "crystal_sz_history": data[7],
}

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)
for line in [
    (
        "crystal_sx_history",
        "blue",
        "Size X",
    ),
    (
        "crystal_sy_history",
        "green",
        "Size Y",
    ),
    (
        "crystal_sz_history",
        "red",
        "Size Z",
    ),
]:
    ax.plot(
        data["time"],
        data[line[0]],
        color=line[1],
        linewidth=1,
        marker="o",
        markersize=3,
        markeredgewidth=1.0,
        markerfacecolor=line[1],
        markeredgecolor="black",
        markevery=1,
        label=line[2],
    )

# ax.set_title(f"Axis Z")
ax.legend(loc="best")
# ax.set_xlabel("State ID")
ax.set_xlabel("Time")
ax.set_ylabel("Size")
ax.grid(True)
# plt.savefig(Path(r"AxisVelocity") / f"AxisX.png")
plt.show()
plt.close()

In [ ]:
# Міра відхилення — на основі R²
# error_metric = lambda y_true, y_pred: np.sqrt(np.sum(np.power(y_true - y_pred, 2)))
# error_metric = lambda y_true, y_pred: np.sum((y_true - y_pred) ** 2)
error_metric = lambda y_true, y_pred: (1.0 - calculate_r2(y_true, y_pred)) ** 2


init_params = [
    # [0.5, 0.5],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.07884803813538599, 423.1661829871642],
]

# Визначення моделей
models = {
    "0": {
        "name": "linear",
        "formula": "c1 * x + c2",
        "format_str": "{0:.5g} * x + {1:.5g}",
        "func": lambda x, c1, c2: c1 * x + c2,
        "error": model_error,
        "init_params": init_params[0],
    },
    # "1": {
    #     "name": "exp",
    #     "formula": "c1 * np.exp(x * c2)",
    #     "format_str": "{0:.5f} * exp(x * {1:.5f})",
    #     "func": lambda x, c1, c2: c1 * np.exp(x * c2),
    #     "error": model_error,
    #     "init_params": init_params[1],
    # },
}


# Виклик апроксимації
X = np.copy(data["time"][:])
Y = np.copy(data["crystal_sy_history"][:])

approximations = approximate(
    np.float64(X[40:-1]), np.float64(Y[40:-1]), models=models, sort_by=-1, reverse=True
)

In [ ]:
for apxID, apx in enumerate(approximations):
    print(
        "ID: |{0:^3}| -- R²: |{1:.8f}| -- FID: |{3:^3}| -- M: |{2:^15}| -- Cs: |{4}|".format(
            apxID, apx[-1], *apx[:2], apx[2:-1]
        )
    )

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

ax.plot(
    X,
    Y,
    color="darkred",
    linewidth=1,
    marker="o",
    markersize=4,
    markeredgewidth=0.50,
    markerfacecolor="lightcoral",
    markeredgecolor="darkred",
    markevery=1,
    label="Measured points\n(marked every 1th point)",
)

apx = approximations[0]
t = np.copy(X)
pred = models[apx[1]]["func"](t, *apx[2:-1])

ax.plot(
    t,
    pred,
    color="springgreen",
    linewidth=2,
    label=f"Model Fit: Y = {models[apx[1]]['format_str'].format(*apx[2:-1])}\nR² = {apx[-1]:.5f}",
)


# ax.set_title("Deviation ( Time )", fontsize=14)
ax.legend(loc="best", fontsize=10)
ax.set_xlabel("Time", fontsize=12)
ax.set_ylabel("Size by Y axis", fontsize=12)
ax.grid(True, which="major", linestyle="--", alpha=0.7)
ax.tick_params(axis="both", which="major", labelsize=10)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
# plt.savefig(path_root / "Growth curve 10-1.png", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Міра відхилення — на основі R²
error_metric = lambda y_true, y_pred: (1.0 - calculate_r2(y_true, y_pred)) ** 2


init_params = [
    # [0.5, 0.5],
    [0.00001, 100],
]

# Визначення моделей
models = {
    "0": {
        "name": "linear",
        "formula": "c1 * x + c2",
        "format_str": "{0:.5g} * x + {1:.5g}",
        "func": lambda x, c1, c2: c1 * x + c2,
        "error": model_error,
        "init_params": init_params[0],
    },
}


base_folder = Path(
    r"NEW 1"
)

results = []

try:
    for root, _, files in os.walk(base_folder):
        if {"InitSettings.ini", "TimeStates.txt"}.issubset(files):
            print(f"Processing directory: {root}")

            path_to_dir = Path(root)

            cfg = load_Cfg_new(path_to_dir / "InitSettings.ini")
            run_id = path_to_dir.name.split("_")[0]
            cfg["id"] = run_id

            data = (
                (path_to_dir / "sim_history.txt")
                .open(mode="r", encoding="utf-8")
                .read()
            )
            data = [
                *map(
                    lambda line: [*map(lambda item: float(item), line.split(":"))],
                    data.splitlines(),
                )
            ]
            data = np.array(data)
            data = {
                "time": np.arange(data.shape[1]) * cfg["WriteI"],
                "n_gas_history": data[0],
                "n_crystal_history": data[1],
                "concentration_history": data[2],
                "delta_gibbs_history": data[3],
                "energy_change_history": data[4],
                "crystal_sx_history": data[5],
                "crystal_sy_history": data[6],
                "crystal_sz_history": data[7],
            }

            fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)
            for line in [
                (
                    "crystal_sx_history",
                    "blue",
                    "Size X",
                ),
                (
                    "crystal_sy_history",
                    "green",
                    "Size Y",
                ),
                (
                    "crystal_sz_history",
                    "red",
                    "Size Z",
                ),
            ]:
                ax.plot(
                    data["time"],
                    data[line[0]],
                    color=line[1],
                    linewidth=1,
                    marker="o",
                    markersize=3,
                    markeredgewidth=1.0,
                    markerfacecolor=line[1],
                    markeredgecolor="black",
                    markevery=1,
                    label=line[2],
                )

            # ax.set_title(f"Axis Z")
            ax.legend(loc="best")
            # ax.set_xlabel("State ID")
            ax.set_xlabel("Time")
            ax.set_ylabel("Size")
            ax.grid(True)
            plt.savefig(path_to_dir / f"SizeByAxisOnTime.png", bbox_inches="tight")
            # plt.show()
            plt.close()

            # Виклик апроксимації
            X = np.copy(data["time"][:])
            Y = np.copy(data["crystal_sy_history"][:])

            approximations = approximate(
                np.float64(X[40:-1]),
                np.float64(Y[40:-1]),
                models=models,
                sort_by=-1,
                reverse=True,
            )

            fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

            ax.plot(
                X,
                Y,
                color="darkred",
                linewidth=1,
                marker="o",
                markersize=4,
                markeredgewidth=0.50,
                markerfacecolor="lightcoral",
                markeredgecolor="darkred",
                markevery=1,
                label="Measured points\n(marked every 1th point)",
            )

            apx = approximations[0]
            t = np.copy(X)
            pred = models[apx[1]]["func"](t, *apx[2:-1])

            ax.plot(
                t,
                pred,
                color="springgreen",
                linewidth=2,
                label=f"Model Fit: Y = {models[apx[1]]['format_str'].format(*apx[2:-1])}\nR² = {apx[-1]:.5f}",
            )

            # ax.set_title("Deviation ( Time )", fontsize=14)
            ax.legend(loc="best", fontsize=10)
            ax.set_xlabel("Time", fontsize=12)
            ax.set_ylabel("Size by Y axis", fontsize=12)
            ax.grid(True, which="major", linestyle="--", alpha=0.7)
            ax.tick_params(axis="both", which="major", labelsize=10)
            ax.spines["top"].set_visible(False)
            ax.spines["right"].set_visible(False)

            plt.tight_layout()
            plt.savefig(
                path_to_dir / f"SizeByAxisYOnTimeAprox.png", bbox_inches="tight"
            )
            # plt.show()
            plt.close()

            results.append([cfg["id"], cfg["p_b"], apx[-1], *apx[2:-1]])


except Exception as e:
    print(f"Error in main loop: {str(e)}")
    raise

results = np.array(results, dtype=float)

In [ ]:
print(results)

In [ ]:
# Міра відхилення — на основі R²
# error_metric = lambda y_true, y_pred: np.sqrt(np.sum(np.power(y_true - y_pred, 2)))
# error_metric = lambda y_true, y_pred: np.sum((y_true - y_pred) ** 2)
error_metric = lambda y_true, y_pred: (1.0 - calculate_r2(y_true, y_pred)) ** 2


init_params = [
    # [0.5, 0.5],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.00001, 4],
    # [1.0, 1.0],
    [0.00001, 100],
]

# Визначення моделей
models = {
    "0": {
        "name": "linear",
        "formula": "c1 * x + c2",
        "format_str": "{0:.5g} * x + {1:.5g}",
        "func": lambda x, c1, c2: c1 * x + c2,
        "error": model_error,
        "init_params": init_params[0],
    },
    "1": {
        "name": "power 4",
        "formula": "c1 * (x ^ c2)",
        "format_str": "{0:.5f} * (x ^ {1:.5f})",
        "func": lambda x, c1, c2: c1 * (x ** c2),
        "error": model_error,
        "init_params": init_params[1],
    },
    "2": {
        "name": "exp",
        "formula": "c1 * exp(x * c2)",
        "format_str": "{0:.5g} * exp(x * {1:.5g})",
        "func": lambda x, c1, c2: c1 * np.exp(x * c2),
        "error": model_error,
        "init_params": init_params[2],
    },
}


# Виклик апроксимації
X = np.copy(results[:11, 1])
Y = np.copy(results[:11, 3])

approximations = approximate(
    np.float64(X[:]), np.float64(Y[:]), models=models, sort_by=-1, reverse=True
)

In [ ]:
for apxID, apx in enumerate(approximations):
    print(
        "ID: |{0:^3}| -- R²: |{1:.8f}| -- FID: |{3:^3}| -- M: |{2:^15}| -- Cs: |{4}|".format(
            apxID, apx[-1], *apx[:2], apx[2:-1]
        )
    )

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

ax.plot(
    X,
    Y,
    color="darkred",
    linewidth=1,
    marker="o",
    markersize=4,
    markeredgewidth=0.50,
    markerfacecolor="lightcoral",
    markeredgecolor="darkred",
    markevery=1,
    label="Measured points\n(marked every 1th point)",
)

apx = approximations[0]
t = np.copy(X)
pred = models[apx[1]]["func"](t, *apx[2:-1])

ax.plot(
    t,
    pred,
    color="springgreen",
    linewidth=2,
    label=f"Model Fit: Y = {models[apx[1]]['format_str'].format(*apx[2:-1])}\nR² = {apx[-1]:.5f}",
)


# ax.set_title("Deviation ( Time )", fontsize=14)
ax.legend(loc="best", fontsize=10)
ax.set_xlabel("Time", fontsize=12)
ax.set_ylabel("Size by Y axis", fontsize=12)
ax.grid(True, which="major", linestyle="--", alpha=0.7)
ax.tick_params(axis="both", which="major", labelsize=10)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
# plt.savefig(path_root / "Growth curve 10-1.png", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

ax.plot(
    results[11:, 1],
    results[11:, 3],
    color="darkred",
    linewidth=1,
    marker="o",
    markersize=4,
    markeredgewidth=0.50,
    markerfacecolor="lightcoral",
    markeredgecolor="darkred",
    markevery=1,
    label="Ntotal = 5e12",
)
ax.plot(
    results[:11, 1],
    results[:11, 3],
    color="darkblue",
    linewidth=1,
    marker="o",
    markersize=4,
    markeredgewidth=0.50,
    markerfacecolor="lightcoral",
    markeredgecolor="darkblue",
    markevery=1,
    label="Ntotal = 1e13",
)

apx = approximations[0]
t = np.copy(X)
pred = models[apx[1]]["func"](t, *apx[2:-1])

# ax.plot(
#     t,
#     pred,
#     color="springgreen",
#     linewidth=2,
#     label=f"Model Fit: Y = {models[apx[1]]['format_str'].format(*apx[2:-1])}\nR² = {apx[-1]:.5f}",
# )


# ax.set_title("Deviation ( Time )", fontsize=14)
ax.legend(loc="best", fontsize=10)
ax.set_xlabel("Time", fontsize=12)
ax.set_ylabel("Size by Y axis", fontsize=12)
ax.grid(True, which="major", linestyle="--", alpha=0.7)
ax.tick_params(axis="both", which="major", labelsize=10)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
# plt.savefig(path_root / "Growth curve 10-1.png", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Міра відхилення — на основі R²
error_metric = lambda y_true, y_pred: (1.0 - calculate_r2(y_true, y_pred)) ** 2


init_params = [
    # [0.5, 0.5],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.00001, 4],
    # [1.0],
    [0.00001],
]

# Визначення моделей
models = {
    "0": {
        "name": "linear",
        "formula": "c1 * x + c2",
        "format_str": "{0:.5g} * x + {1:.5g}",
        "func": lambda x, c1, c2: c1 * x + c2,
        "error": model_error,
        "init_params": init_params[0],
    },
    "1": {
        "name": "exp",
        "formula": "c1 * exp(x * c2)",
        "format_str": "{0:.5g} * exp(x * {1:.5g})",
        "func": lambda x, c1, c2: c1 * np.exp(x * c2),
        "error": model_error,
        "init_params": init_params[1],
    },
    "2": {
        "name": "power 4",
        "formula": "c1 * (x ^ c2)",
        "format_str": "{0:.5f} * (x ^ {1:.5f})",
        "func": lambda x, c1, c2: c1 * (x ** c2),
        "error": model_error,
        "init_params": init_params[2],
    },
    "3": {
        "name": "power 4",
        "formula": "c1 * (x ^ 4.0)",
        "format_str": "{0:.5f} * (x ^ 4.0)",
        "func": lambda x, c1: c1 * (x ** 4.0),
        "error": model_error,
        "init_params": init_params[3],
    },
}


results1 = []
for x, y in [[results[:11, 1], results[:11, 3]], [results[11:, 1], results[11:, 3]]]:
    X = np.copy(x)
    Y = np.copy(y)

    approximations = approximate(
        np.float64(X[:]), np.float64(Y[:]), models=models, sort_by=-1, reverse=True
    )

    results1.append([X, Y, approximations])

In [ ]:
for res1 in results1:
    for apxID, apx in enumerate(res1[2]):
        print(
            "ID: |{0:^3}| -- R²: |{1:.8f}| -- FID: |{3:^3}| -- M: |{2:^15}| -- Cs: |{4}|".format(
                apxID, apx[-1], *apx[:2], apx[2:-1]
            )
        )

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 5,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 3,
    }
)
# Під розмір стовпця журналу (~8.9 см)
fig, ax = plt.subplots(figsize=(1.0 * 3.5, 1.0 * 2.0), dpi=600)

apx0 = results1[0][2][0]
t0 = np.copy(results1[0][0])
pred0 = models[apx0[1]]["func"](t0, *apx0[2:-1])

apx1 = results1[1][2][0]
t1 = np.copy(results1[1][0])
pred1 = models[apx1[1]]["func"](t1, *apx1[2:-1])

datasets = [
    (results[:11, 1], results[:11, 3], "s", "", "black", 4, "Ntotal = 5e12"),
    (results[11:, 1], results[11:, 3], "o", "", "black", 3, "Ntotal = 1e13"),
    (
        t0,
        pred0,
        "s",
        (0, (5, 5)),
        "black",
        1.5,
        f"Model Fit: Y = {models[apx0[1]]['format_str'].format(*apx0[2:-1])}\nR² = {apx0[-1]:.5f}",
    ),
    (
        t1,
        pred1,
        "o",
        (0, (5, 10)),
        "dimgray",
        1.5,
        f"Model Fit: Y = {models[apx1[1]]['format_str'].format(*apx1[2:-1])}\nR² = {apx1[-1]:.5f}",
    ),
]

for x, y, marker, linestyle, color, msize, label in datasets:
    ax.plot(
        x,
        y,
        marker=marker,
        linestyle=linestyle,
        color=color,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markersize=msize,
        markevery=1,
        label=label,
    )


ax.set_xlabel("Ballistic probability")
ax.set_ylabel("Elongation Velocity")
ax.grid(True, which="both", linestyle=":", linewidth=0.4)
ax.tick_params(direction="in", length=3, width=0.5)

# Змінені параметри для легенди:
# ax.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.0, frameon=False)
# bbox_to_anchor=(1.02, 1):
#   - 1.01 означає, що лівий край легенди буде на 1% ширини осі справа від правого краю осі.
#   - 1 означає, що верхній край легенди буде вирівняний по верхньому краю осі.
# loc='upper left': Верхній лівий кут легенди прив'язується до точки (1.01, 1).
# borderaxespad=0.: Прибирає відступ між рамкою осей та легендою.
ax.legend(loc="upper left", borderaxespad=0.0, frameon=False)

plt.tight_layout()  # Це дуже важливо! Він автоматично коригує розміри графіка, щоб легенда помістилася.

# plt.savefig(
#     Path(r"AxisVelocity")
#     / "Axes.png",
#     format="png",
# )
plt.show()
plt.close()

In [ ]:
# Міра відхилення — на основі R²
error_metric = lambda y_true, y_pred: (1.0 - calculate_r2(y_true, y_pred)) ** 2


init_params = [
    # [0.5, 0.5],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.00001, 100],
    # [1.0, 1.0],
    [0.00001, 4],
    # [1.0],
    [0.00001],
]

# Визначення моделей
models = {
    "0": {
        "name": "linear",
        "formula": "c1 * x + c2",
        "format_str": "{0:.5g} * x + {1:.5g}",
        "func": lambda x, c1, c2: c1 * x + c2,
        "error": model_error,
        "init_params": init_params[0],
    },
    "1": {
        "name": "exp",
        "formula": "c1 * exp(x * c2)",
        "format_str": "{0:.5g} * exp(x * {1:.5g})",
        "func": lambda x, c1, c2: c1 * np.exp(x * c2),
        "error": model_error,
        "init_params": init_params[1],
    },
    "2": {
        "name": "power 4",
        "formula": "c1 * (x ^ c2)",
        "format_str": "{0:.5f} * (x ^ {1:.5f})",
        "func": lambda x, c1, c2: c1 * (x ** c2),
        "error": model_error,
        "init_params": init_params[2],
    },
    "3": {
        "name": "power 4",
        "formula": "c1 * (x ^ 4.0)",
        "format_str": "{0:.5f} * (x ^ 4.0)",
        "func": lambda x, c1: c1 * (x ** 4.0),
        "error": model_error,
        "init_params": init_params[3],
    },
}


results2 = []
for x, y in [[results[:11, 1], results[:11, 3]], [results[11:, 1], results[11:, 3]]]:
    X = np.copy(x)
    Y = np.log(np.copy(y))

    approximations = approximate(
        np.float64(X[:]), np.float64(Y[:]), models=models, sort_by=-1, reverse=True
    )

    results2.append([X, Y, approximations])

In [ ]:
for res1 in results2:
    for apxID, apx in enumerate(res1[2]):
        print(
            "ID: |{0:^3}| -- R²: |{1:.8f}| -- FID: |{3:^3}| -- M: |{2:^15}| -- Cs: |{4}|".format(
                apxID, apx[-1], *apx[:2], apx[2:-1]
            )
        )

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 5,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 3,
    }
)
# Під розмір стовпця журналу (~8.9 см)
fig, ax = plt.subplots(figsize=(1.0 * 3.5, 1.0 * 2.0), dpi=600)

apx0 = results2[0][2][0]
t0 = np.copy(results2[0][0])
pred0 = models[apx0[1]]["func"](t0, *apx0[2:-1])

apx1 = results2[1][2][0]
t1 = np.copy(results2[1][0])
pred1 = models[apx1[1]]["func"](t1, *apx1[2:-1])

datasets = [
    (results2[0][0], results2[0][1], "s", "", "black", 4, "Ntotal = 5e12"),
    (results2[1][0], results2[1][1], "o", "", "black", 3, "Ntotal = 1e13"),
    (
        t0,
        pred0,
        "s",
        (0, (5, 5)),
        "black",
        1.5,
        f"Model Fit: Y = {models[apx0[1]]['format_str'].format(*apx0[2:-1])}\nR² = {apx0[-1]:.5f}",
    ),
    (
        t1,
        pred1,
        "o",
        (0, (5, 10)),
        "dimgray",
        1.5,
        f"Model Fit: Y = {models[apx1[1]]['format_str'].format(*apx1[2:-1])}\nR² = {apx1[-1]:.5f}",
    ),
]

for x, y, marker, linestyle, color, msize, label in datasets:
    ax.plot(
        x,
        y,
        marker=marker,
        linestyle=linestyle,
        color=color,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markersize=msize,
        markevery=1,
        label=label,
    )


ax.set_xlabel("Ballistic probability")
ax.set_ylabel("LN ( Elongation Velocity )")
ax.grid(True, which="both", linestyle=":", linewidth=0.4)
ax.tick_params(direction="in", length=3, width=0.5)

# Змінені параметри для легенди:
# ax.legend(bbox_to_anchor=(1.01, 1), loc="upper left", borderaxespad=0.0, frameon=False)
# bbox_to_anchor=(1.02, 1):
#   - 1.01 означає, що лівий край легенди буде на 1% ширини осі справа від правого краю осі.
#   - 1 означає, що верхній край легенди буде вирівняний по верхньому краю осі.
# loc='upper left': Верхній лівий кут легенди прив'язується до точки (1.01, 1).
# borderaxespad=0.: Прибирає відступ між рамкою осей та легендою.
ax.legend(loc="upper left", borderaxespad=0.0, frameon=False)

plt.tight_layout()  # Це дуже важливо! Він автоматично коригує розміри графіка, щоб легенда помістилася.

# plt.savefig(
#     Path(r"AxisVelocity")
#     / "Axes.png",
#     format="png",
# )
plt.show()
plt.close()